# Create dataset from wide to long format

In [1]:
# imports
import pandas as pd

In [2]:
# local imports
from src.d00_helpers import helpers as h

In [3]:
# read in dataset
name='2024.04.24_Mastertabelle'
df = pd.read_excel(f'G://My Drive//Forschung//Mitarbeiter/Allgaier//23-12-06_Immun-ML//04_Data//00_raw//{name}.xlsx',
                   na_values=[' ',''])

In [4]:
# read in codebook
abs_path_to_codebook = 'G://My Drive//Forschung//Mitarbeiter/Allgaier//23-12-06_Immun-ML//01_Codebook//Codebook.xlsx'
cb = pd.read_excel(abs_path_to_codebook, sheet_name='TimePoints').iloc[:25]

In [5]:
# define new columns in long format
new_cols = ['ID', 'date', 'timepoint', 'vaccination', 'infection', 'SARS-IgG']
df_long = pd.DataFrame(columns = new_cols)

In [6]:
# create mappings from T-timestamps (T1, T2, ..., T34) to vaccination and from T-timestamps to dates

# Which T-timestamps refers to which vaccination number?
t_date_vaccs = dict(zip(cb['coding'], cb['relation_to_number_of_vaccination']))

# What is the relative difference in number of date to vaccination date?
t_date_mapping = h.create_T_date_mapping(abs_path_to_codebook)

In [7]:
# get a list of all patient IDs
patient_ids = [id for id in df['ID'].unique()]

# which variables shall be in long format?
measurement_column = 'SARS-IgG'

cols = ['patient_id', 'timepoint', measurement_column, 'date', 'vaccination', 'infection']
res = pd.DataFrame(columns=cols)

for patient_id in patient_ids:
    res_p = h.calculate_measuring_date_and_create_long_format(df, t_date_mapping, t_date_vaccs,patient_id, measurement_column)
    res = pd.concat([res if not res.empty else None, res_p], ignore_index=True)

res['infection'] = res['infection'].fillna(0)
res['vaccination'] = res['vaccination'].fillna(0)

Patient without any date:  C327
Patient without any date:  C361
Patient without any date:  C362
Patient without any date:  C363
Patient without any date:  C364
Patient without any date:  C365
Patient without any date:  C366
Patient without any date:  C367
Patient without any date:  C368
Patient without any date:  C369
Patient without any date:  C370


In [8]:
res.to_excel(f'G://My Drive//Forschung//Mitarbeiter/Allgaier//23-12-06_Immun-ML//04_Data//01_processed//{name}_long.xlsx')